In [1]:
import torch

from gfn.envs import HyperGrid
from gfn import LogitPBEstimator, LogitPFEstimator, LogZEstimator
from gfn.losses import TBParametrization, TrajectoryBalance
from gfn.samplers import DiscreteActionsSampler, TrajectoriesSampler


torch.manual_seed(0)

exploration_rate = 0.5
learning_rate=0.0005

env = HyperGrid(ndim=5, height=2)

logit_PF = LogitPFEstimator(
    env=env,
    module_name="NeuralNet",
)
logit_PB = LogitPBEstimator(
    env=env,
    module_name="NeuralNet",
    torso=logit_PF.module.torso,
)
logZ = LogZEstimator(torch.tensor(0.0))

training_sampler = TrajectoriesSampler(
    env=env,
    actions_sampler=DiscreteActionsSampler(
        estimator=logit_PF,
        epsilon=exploration_rate
    )
)

parametrization = TBParametrization(logit_PF, logit_PB, logZ)
loss_fn = TrajectoryBalance(
    parametrization=parametrization,
)

params = [
    {
        "params": [
            val for key, val in parametrization.parameters.items() if "logZ" not in key
        ],
        "lr": learning_rate,
    },
    {"params": [val for key, val in parametrization.parameters.items() if "logZ" in key], "lr": 0.1},
]
optimizer = torch.optim.Adam(params=params)




/home/josephb/anaconda3/envs/gfn/lib/python3.10/site-packages/torch/optim/adam.py:33: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  super().__init__(params, defaults)


In [2]:
from tqdm import tqdm


n_iterations = int(1e4)
batch_size = int(1e5)

for i in (pbar := tqdm(range(n_iterations))):
    trajectories = training_sampler.sample(
        n_trajectories=batch_size
    )

    optimizer.zero_grad()
    loss = loss_fn(trajectories)
    loss.backward()
    optimizer.step()

    if i % int(1e4) == 0:
        pbar.set_postfix({"loss": loss.item()})

  0%|                                                                                                                                             | 0/10000 [00:00<?, ?it/s]/home/josephb/anaconda3/envs/gfn/lib/python3.10/site-packages/torch/autograd/__init__.py:200: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
  0%|▍                                                                                                                     | 40/10000 [02:38<10:59:15,  3.97s/it, loss=7.66]


NonValidActionsError: Actions are not valid